In [80]:
import pymongo
from bson.son import SON
from pprint import pprint

In [2]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db

In [17]:
# Insert one item
db.inventory.insert_one(
    {"item": "canvas",
     "qty": 100,
     "status": "C",
     "tags": ["cotton"],
     "size": {"h": 28, "w": 35.5, "uom": "cm"},
     "dim_cm": [28, 35.5]})

In [113]:
# Insert many items
db.inventory.delete_many({})

db.inventory.insert_many([
   # MongoDB adds the _id field with an ObjectId if _id is not present
   { 
       "item": "journal", 
       "qty": 25, 
       "status": "A",
       "size": { 
           "h": 14, 
           "w": 21, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "notebook", 
       "qty": 50, 
       "status": "A",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank" ],
       "dim_cm": [14, 21]
   },
   { 
       "item": "paper", 
       "qty": 100, 
       "status": "D",
       "size": { 
           "h": 8.5, 
           "w": 11, 
           "uom": "in" 
       }, 
       "tags": [ "red", "blank", "plain" ],
       "dim_cm": [8.5, 11]
   },
   { 
       "item": "planner", 
       "qty": 75, 
       "status": "D",
       "size": { 
           "h": 22.85, 
           "w": 30, 
           "uom": "cm" 
       }, 
       "tags": [ "blank", "red" ] ,
       "dim_cm": [22.85, 30]
   },
   { 
       "item": "postcard", 
       "qty": 45, 
       "status": "A",
       "size": 
       { 
           "h": 10, 
           "w": 15.25, 
           "uom": "cm" 
       }, 
       "tags": [ "blue" ],
       "dim_cm": [10, 15.25]
   }
])

In [114]:
# Query all documents
for item in db.inventory.find({}):
    print(item['item'])

journal
notebook
paper
planner
postcard


In [41]:
# Query exact matchs with dictionary
for item in db.inventory.find({'size' : { "h": 10, "w": 15.25, "uom": "cm" }}):
    print(item['item'], item['size'])

postcard {'h': 10, 'w': 15.25, 'uom': 'cm'}


In [40]:
# Query subdocument items
for item in db.inventory.find({"size.uom": "in"}):
    print(item['item'], item['size']['uom'])

notebook in
paper in


In [38]:
# SELECT * FROM inventory WHERE size.h

for item in db.inventory.find({'size.h': {'$gt' : 10 }}):
    print(item['item'], item['size']['h'])

journal 14
planner 22.85
canvas 28


In [35]:
# SELECT * FROM inventory WHERE status in ("A", "D")
for item in db.inventory.find({'status': {"$in" : ['A', 'D']}}):
    print(item['item'], item['status'])

journal A
notebook A
paper D
planner D
postcard A


In [34]:
# SELECT * FROM inventory WHERE status = 'D' AND qty > 50
for item in db.inventory.find({'status' : 'D', 'qty' : {'$gt' : 50 }}):
    print(item['item'], item['status'], item['qty'])

paper D 100
planner D 75


In [11]:
# SELECT * FROM inventory WHERE status = 'C' AND qty < 50
for item in db.inventory.find({'$or' : [{'status' : 'D'}, {'qty': {'$lt' : 50}}]}):
    print(item['status'], item['qty'])
                                        
                                        

A 25
D 100
D 75
A 45


In [12]:
# SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")
cursor = db.inventory.find({
    "status": "A",
    "$or": [{"qty": {"$lt": 30}}, {"item": {"$regex": "^p"}}]})

for item in cursor:
    print(item['status'], item['qty'], item['item'])

A 25 journal
A 45 postcard


In [33]:
# Exact maching
for item in db.inventory.find({"tags": ["red", "blank"]}):
    print(item['item'], item["tags"])


notebook ['red', 'blank']


In [32]:
# Query arrays which contains "red" and "black" in any order

for item in db.inventory.find({"tags": { '$all' : ["red", "blank"]}}):
    print(item['item'], item["tags"])


journal ['blank', 'red']
notebook ['red', 'blank']
paper ['red', 'blank', 'plain']
planner ['blank', 'red']


In [31]:
# Query arrays which contains "red"

for item in db.inventory.find({"tags": "red"}):
    print(item['item'], item["tags"])

journal ['blank', 'red']
notebook ['red', 'blank']
paper ['red', 'blank', 'plain']
planner ['blank', 'red']


In [30]:
#  Queries for all documents where the array dim_cm contains at least one element whose value is greater than 25.
for item in db.inventory.find({"dim_cm": {'$gt' : 25}}):
    print(item['item'], item["dim_cm"])

planner [22.85, 30]
canvas [28, 35.5]


In [51]:
# 15보다 작은 하나의 엘리먼트가 하나 이상, 20보다 큰 엘리먼트가 하나 이상 있어야 한다.
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])
print('------')    
for item in db.inventory.find({"dim_cm": {"$lt": 15, "$gt": 20}}):
    print(item['item'], item["dim_cm"])

journal [14, 21]
notebook [14, 21]
paper [8.5, 11]
planner [22.85, 30]
postcard [10, 15.25]
canvas [28, 35.5]
------
journal [14, 21]
notebook [14, 21]


In [60]:
# 'Or' condition
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------')  

for item in db.inventory.find({'$or' : [{"dim_cm": {"$lt": 9}}, {"dim_cm": {"$gt": 20}}]}):
    print(item['item'], item["dim_cm"])

journal [14, 21]
notebook [14, 21]
paper [8.5, 11]
planner [22.85, 30]
postcard [10, 15.25]
canvas [28, 35.5]
------
journal [14, 21]
notebook [14, 21]
paper [8.5, 11]
planner [22.85, 30]
canvas [28, 35.5]


In [61]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# dim_cm에 두 조건을 모두 만족하는 엘리먼트가 하나 이상 있어야한다.
for item in db.inventory.find({"dim_cm": {"$elemMatch": {"$gt": 22, "$lt": 30}}}):
    print(item['item'], item["dim_cm"])

journal [14, 21]
notebook [14, 21]
paper [8.5, 11]
planner [22.85, 30]
postcard [10, 15.25]
canvas [28, 35.5]
------
planner [22.85, 30]
canvas [28, 35.5]


In [65]:
for item in db.inventory.find({}):
    print(item['item'], item["dim_cm"])

print('------') 

# INDEX를 이용한 접근 방법
for item in db.inventory.find({'dim_cm.1': {"$gt": 25}}):
    print(item['item'], item['dim_cm'])

journal [14, 21]
notebook [14, 21]
paper [8.5, 11]
planner [22.85, 30]
postcard [10, 15.25]
canvas [28, 35.5]
------
planner [22.85, 30]
canvas [28, 35.5]


In [73]:
# Array 길이
# $gt를 사용해서 2보다 큰 Array 모두 반환하는 그런 문법은 안되는 듯.
for item in db.inventory.find({'tags' : {'$size': 2}}):
    print(item['item'], item['tags'])

journal ['blank', 'red']
notebook ['red', 'blank']
planner ['blank', 'red']


In [78]:
# 1보다 큰 Array는 아래 처럼 사용하면 됨
for item in db.inventory.find({'$where' : 'this.tags.length > 1'}):
    print(item['item'], item['tags'])

journal ['blank', 'red']
notebook ['red', 'blank']
paper ['red', 'blank', 'plain']
planner ['blank', 'red']


In [81]:
# 서브 document가 배열 형태로 있는 예제를 수행하기 위해 새로운 DB생성
db.inventory.delete_many({})

db.inventory.insert_many([
    {"item": "journal",
     "instock": [
         SON([("warehouse", "A"), ("qty", 5)]),
         SON([("warehouse", "C"), ("qty", 15)])]},
    {"item": "notebook",
     "instock": [
         SON([("warehouse", "C"), ("qty", 5)])]},
    {"item": "paper",
     "instock": [
         SON([("warehouse", "A"), ("qty", 60)]),
         SON([("warehouse", "B"), ("qty", 15)])]},
    {"item": "planner",
     "instock": [
         SON([("warehouse", "A"), ("qty", 40)]),
         SON([("warehouse", "B"), ("qty", 5)])]},
    {"item": "postcard",
     "instock": [
         SON([("warehouse", "B"), ("qty", 15)]),
         SON([("warehouse", "C"), ("qty", 35)])]}])

In [91]:
# 리스트 서브 문서 중 하나의 문서라도 조건을 만족하면 반환

for item in db.inventory.find({"instock": SON([("warehouse", "A"), ("qty", 5)])}):
    print(item['item'], item['instock'])
    
print('필드 위치 바뀌면 안된다.')    

for item in db.inventory.find({"instock": SON([("qty", 5), ("warehouse", "A")])}):
    print(item['item'], item['instock'])

journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
필드 위치 바뀌면 안된다.


In [101]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('- 하나의 문서라도 less than equal에 걸리면 반환')

for item in db.inventory.find({'instock.qty': {"$lte": 5}}):
    print(item['item'], item['instock'])

journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
notebook [{'warehouse': 'C', 'qty': 5}]
paper [{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]
planner [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]
postcard [{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]
하나의 문서라도 less than equal에 걸리면 반환
journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
notebook [{'warehouse': 'C', 'qty': 5}]
planner [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]


In [107]:
for item in db.inventory.find({}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 두 조건을 모두 만족해야 함')

for item in db.inventory.find({"instock": {"$elemMatch": {"qty": {"$gt": 10, "$lte": 20}}}}):
    print(item['item'], item['instock'])

print('-각 엘리먼트의 qty가 아래 한 조건만 맞는 엘리만트가 있으면 반환')
    
for item in db.inventory.find({"instock.qty": {"$lt": 10, "$lte": 30}}):
    print(item['item'], item['instock'])
    
    

journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
notebook [{'warehouse': 'C', 'qty': 5}]
paper [{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]
planner [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]
postcard [{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]
-각 엘리먼트의 qty가 두 조건을 모두 만족해야 함
journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
paper [{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}]
postcard [{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}]
-각 엘리먼트의 qty가 아래 한 조건만 맞는 엘리만트가 있으면 반환
journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
notebook [{'warehouse': 'C', 'qty': 5}]
planner [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]


In [108]:
for item in db.inventory.find({"instock.qty": 5, "instock.warehouse": "A"}):
    print(item['item'], item['instock'])

journal [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}]
planner [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}]


In [115]:
db.inventory.delete_many({})

db.inventory.insert_many([
    {
        "item": "journal",
        "status": "A",
        "size": {"h": 14, "w": 21, "uom": "cm"},     
        "instock": [{"warehouse": "A", "qty": 5}]
    },
    {
        "item": "notebook",
        "status": "A",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "C", "qty": 5}]
    },
    {
        "item": "paper",
        "status": "D",
        "size": {"h": 8.5, "w": 11, "uom": "in"},
        "instock": [{"warehouse": "A", "qty": 60}]
    }, 
    {
        "item": "planner",
        "status": "D",
        "size": {"h": 22.85, "w": 30, "uom": "cm"},
        "instock": [{"warehouse": "A", "qty": 40}]
    },
    {
        "item": "postcard",
        "status": "A",
        "size": {"h": 10, "w": 15.25, "uom": "cm"},
        "instock": [
            {"warehouse": "B", "qty": 15},
            {"warehouse": "C", "qty": 35}
        ]
    }
])


In [118]:
for item in db.inventory.find({"status": "A"}):
    print(item['item'], item['status'])


journal A
notebook A
postcard A
